In [2]:
# -*- coding: utf-8
# VOLT DATA LAB (https://www.voltdata.info/)
# Reinaldo Chaves (@paidatocandeira)
# Detalhamento de como é o gasto com funcionários em cada gabinete e detecta as falhas de transparência do sistema 
#

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import unidecode

Função para retirar acentuação

In [6]:
def f(str):
    return (unidecode.unidecode(str))

Formato do float

In [7]:
pd.options.display.float_format = '{:,.2f}'.format

# Remuneração de servidores da Alesp

Fonte: https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2018-07.html <br>
Último mẽs com informações - julho de 2018

In [8]:
url_chave = ' https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-'
pagamentos = []

Raspa os vencimentos a partir de 2015

In [9]:
for vez in [2015, 2016, 2017, 2018]:
    ano = str(vez)
    for i in range(1, 13):
        if ano == '2018' and i == 8:
            break
        else:
            if i <= 9:
                mes = '0' +  str(i)
            else:
                mes = str(i)
                
            
            link = url_chave + ano + '-' + mes + '.html'
            print(link)
            
            url = urlopen(link)
            
            bsObje = BeautifulSoup(url, "html5lib")
            
            tabela = bsObje.findAll("table", {"class":"tabela_simples"})
            
            conta = 0

            for table_ele in tabela:
                for row in table_ele.findAll('tr'):
                    if conta > 0:
                        cols = row.findAll('td')
                        dicionario = {"ano": ano,
                                      "mes": mes,
                                      "nome_func": cols[0].text.strip(), 
                                      "remuneracao_bruta": cols[1].text.strip(),
                                      "remuneracao_liquida": cols[2].text.strip(),
                                      "tributos": cols[3].text.strip()}
                        pagamentos.append(dicionario)         
                    conta = conta + 1           

 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-01.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-02.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-03.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-04.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-05.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-06.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-07.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-08.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-09.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-10.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-11.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2015-12.html
 https://www.al.sp.gov.br/repositorio/folha-de-pagamento/folha-2016-01.html
 https://www

In [12]:
df_pagamentos = pd.DataFrame(pagamentos)

In [13]:
df_pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211945 entries, 0 to 211944
Data columns (total 6 columns):
ano                    211945 non-null object
mes                    211945 non-null object
nome_func              211945 non-null object
remuneracao_bruta      211945 non-null object
remuneracao_liquida    211945 non-null object
tributos               211945 non-null object
dtypes: object(6)
memory usage: 9.7+ MB


In [14]:
df_pagamentos.reset_index().head()

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos
0,0,2015,01,ABEL PEREIRA,"13.406,63","10.439,20","2.967,43"
1,1,2015,01,ABENOR GARCIA FERREIRA,"17.507,31","13.968,74","3.538,57"
2,2,2015,01,ABILIO DO LIVRAMENTO P ALVES,"11.160,56","7.919,77","3.240,79"
3,3,2015,01,ABNEIA BUFFOLIN BROCANELLI,"4.599,47","3.758,26","841,21"
4,4,2015,01,ABRAO BARBOSA DIB,"9.254,85","7.202,63","2.052,22"


In [15]:
df_pagamentos["nome_func"] = df_pagamentos["nome_func"].apply(f)

In [16]:
df_pagamentos.to_excel('resultados/pagamentos_servidores.xlsx',sheet_name='Sheet1')

In [4]:
df_pagamentos = pd.read_excel('resultados/pagamentos_servidores.xlsx',sheet_name='Sheet1')

In [17]:
df_pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211945 entries, 0 to 211944
Data columns (total 6 columns):
ano                    211945 non-null object
mes                    211945 non-null object
nome_func              211945 non-null object
remuneracao_bruta      211945 non-null object
remuneracao_liquida    211945 non-null object
tributos               211945 non-null object
dtypes: object(6)
memory usage: 9.7+ MB


# Lotação de servidores

Fonte: https://www.al.sp.gov.br/geral/siga/servidores.jsp?todos=true

In [18]:
url = urlopen("https://www.al.sp.gov.br/geral/siga/servidores.jsp?todos=true")
bsObje =  BeautifulSoup(url, "html5lib")

In [19]:
tabela = bsObje.findAll("table", {"class":"tabela lotacao"})

In [20]:
funcionarios = []
i = 0
for table_ele in tabela:
    for row in table_ele.findAll('tr'):
        if i > 1:
            cols = row.findAll('td')
            dicionario = {"nome_func": cols[0].text.strip(), 
                              "tipo_cargo1": cols[1].text.strip(),
                              "tipo_cargo2": cols[2].text.strip(),
                              "lotacao": cols[3].text.strip()}
            funcionarios.append(dicionario)         
        i = i + 1

In [21]:
df_lotacao = pd.DataFrame(funcionarios)

In [22]:
df_lotacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27933 entries, 0 to 27932
Data columns (total 4 columns):
lotacao        27933 non-null object
nome_func      27933 non-null object
tipo_cargo1    27933 non-null object
tipo_cargo2    27933 non-null object
dtypes: object(4)
memory usage: 873.0+ KB


In [23]:
df_lotacao.reset_index().head()

,index,lotacao,nome_func,tipo_cargo1,tipo_cargo2
0,0,GABINETE DEP. ENIO TATTO,ABALDINO FERREIRA DA SILVA,ASSISTENTE PARLAMENTAR II,COMISSÃO
1,1,Ex-funcionário,ABDALLAH ABRÃO AUAD,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
2,2,Ex-funcionário,ABDIAS CRISPIM DE ALMEIDA,,
3,3,Ex-funcionário,ABDIAS DA SILVA,AUXILIAR PARLAMENTAR\n\t\t\t\t\t\t\t\n\t\t\t\t...,COMISSÃO
4,4,Ex-funcionário,ABDULNASSER MOHAMAD ABOU NASSIF,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO


In [24]:
df_lotacao["nome_func"] = df_lotacao["nome_func"].apply(f)

In [25]:
df_lotacao.reset_index().head(10)

,index,lotacao,nome_func,tipo_cargo1,tipo_cargo2
0,0,GABINETE DEP. ENIO TATTO,ABALDINO FERREIRA DA SILVA,ASSISTENTE PARLAMENTAR II,COMISSÃO
1,1,Ex-funcionário,ABDALLAH ABRAO AUAD,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
2,2,Ex-funcionário,ABDIAS CRISPIM DE ALMEIDA,,
3,3,Ex-funcionário,ABDIAS DA SILVA,AUXILIAR PARLAMENTAR\n\t\t\t\t\t\t\t\n\t\t\t\t...,COMISSÃO
4,4,Ex-funcionário,ABDULNASSER MOHAMAD ABOU NASSIF,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
5,5,Ex-funcionário,ABEDIAS PEREIRA DE ARAUJO,,
6,6,GABINETE DEP. CAUÊ MACRIS,ABEL CASTILHO,COMISSIONADOS,COMISSIONAMENTO SEM PREJUÍZO
7,7,Ex-funcionário,ABEL GARCIA DE SOUZA,,
8,8,Ex-funcionário,ABEL HILARION FERNANDEZ JUNIOR,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
9,9,Ex-funcionário,ABEL PEREIRA,,


In [26]:
df_lotacao.to_excel('resultados/lotacao_servidores.xlsx',sheet_name='Sheet1')

In [6]:
df_lotacao = pd.read_excel('resultados/lotacao_servidores.xlsx',sheet_name='Sheet1')

In [27]:
df_lotacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27933 entries, 0 to 27932
Data columns (total 4 columns):
lotacao        27933 non-null object
nome_func      27933 non-null object
tipo_cargo1    27933 non-null object
tipo_cargo2    27933 non-null object
dtypes: object(4)
memory usage: 873.0+ KB


Número de funcionários por lotação

In [28]:
df_lotacao['lotacao'].value_counts()

Ex-funcionário                                         24090
LIDERANÇA DO PSDB                                        119
ASSISTÊNCIA POLICIAL MILITAR                              91
1ª SECRETARIA                                             65
SERVIÇO DE CONTROLE DE FROTA                              60
2ª SECRETARIA                                             58
DIVISÃO DE TRANSPORTES                                    52
NAE - NÚCLEO DE AVALIAÇÃO ESTRATÉGICA                     51
LIDERANÇA DO PT                                           49
LIDERANÇA DO PTB                                          47
GABINETE DEP. LUIZ TURCO                                  33
GABINETE DEP. ANA DO CARMO                                32
GABINETE DEP. TEONILIO BARBA                              32
GABINETE DEP. GILENO GOMES                                31
GABINETE DEP. ENIO TATTO                                  31
GABINETE DEP. PROFESSOR AURIEL                            30
GABINETE DEP. CORONEL CA

Seleciona apenas Gabinete

In [30]:
str_choice = "GABINETE" 
gabinetes = df_lotacao[df_lotacao['lotacao'].\
                       str.contains(str_choice, na=False)]

In [31]:
gabinetes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2351 entries, 0 to 27925
Data columns (total 4 columns):
lotacao        2351 non-null object
nome_func      2351 non-null object
tipo_cargo1    2351 non-null object
tipo_cargo2    2351 non-null object
dtypes: object(4)
memory usage: 91.8+ KB


Funcionários por gabinete

In [33]:
gabinetes['lotacao'].value_counts()

GABINETE DEP. LUIZ TURCO                   33
GABINETE DEP. ANA DO CARMO                 32
GABINETE DEP. TEONILIO BARBA               32
GABINETE DEP. GILENO GOMES                 31
GABINETE DEP. ENIO TATTO                   31
GABINETE DEP. ANALICE FERNANDES            30
GABINETE DEP. LUIZ FERNANDO T. FERREIRA    30
GABINETE DEP. ALENCAR SANTANA BRAGA        30
GABINETE DEP. CORONEL CAMILO               30
GABINETE DEP. PROFESSOR AURIEL             30
GABINETE DEP. GERALDO CRUZ                 29
GABINETE DEP. CÁSSIO NAVARRO               29
GABINETE DEP. CARLOS NEDER                 29
GABINETE DEP. GIL LANCASTER                29
GABINETE DEP. CAUÊ MACRIS                  29
GABINETE DEP. GILMAR GIMENES               29
GABINETE 48                                29
GABINETE DEP. CEZINHA DE MADUREIRA         28
GABINETE DEP. MARCO VINHOLI                28
GABINETE DEP. CAIO FRANÇA                  28
GABINETE DEP. MARCOS DAMASIO               28
GABINETE DEP. LUIZ CARLOS GONDIM  

In [34]:
gabinetes.reset_index().head(10)

,index,lotacao,nome_func,tipo_cargo1,tipo_cargo2
0,0,GABINETE DEP. ENIO TATTO,ABALDINO FERREIRA DA SILVA,ASSISTENTE PARLAMENTAR II,COMISSÃO
1,6,GABINETE DEP. CAUÊ MACRIS,ABEL CASTILHO,COMISSIONADOS,COMISSIONAMENTO SEM PREJUÍZO
2,29,GABINETE DEP. CELSO NASCIMENTO,ABNEIA BUFFOLIN BROCANELLI,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
3,31,GABINETE DEP. CELSO NASCIMENTO,ABNER REISES BARBOSA,ASSISTENTE PARLAMENTAR I,COMISSÃO
4,33,GABINETE DEP. RODRIGO MORAES,ABRAAO MOREIRA DA SILVA,ASSISTENTE PARLAMENTAR II,COMISSÃO
5,34,GABINETE DEP. SEBASTIÃO SANTOS,ABRAHAO RICARDO HACKME,ASSISTENTE PARLAMENTAR IV\n\t\t\t\t\t\t\t\n\t\...,COMISSÃO
6,42,GABINETE DEP. CORONEL CAMILO,ACLERISTON PEREIRA MARTINS,ASSISTENTE PARLAMENTAR IV,COMISSÃO
7,70,GABINETE DEP. ANALICE FERNANDES,ADALTO BATISTA DA SILVA,ASSISTENTE PARLAMENTAR II,COMISSÃO
8,100,GABINETE DEP. GUSTAVO PETTA,ADECIR MENDES FONSECA,ASSISTENTE PARLAMENTAR VII,COMISSÃO
9,151,GABINETE DEP. CLÉLIA GOMES,ADEMAR RAYMUNDO GARCIA,ASSISTENTE PARLAMENTAR II,COMISSÃO


In [35]:
len(gabinetes['nome_func'].value_counts())

2348

In [36]:
gabinetes.to_excel('resultados/funcionarios_gabinetes.xlsx',sheet_name='Sheet1')

# Aqui um problema: há funcionários homônimos<br>
# A Alesp não divulga um código identificador único dos servidores

In [37]:
gabinetes['nome_func'].value_counts()

JOAO CARLOS FRANCISCO DA SILVA            2
ANA MARIA DA SILVA                        2
PAULO EDUARDO ALVES FERREIRA              2
DENISE APARECIDA EUGENIO                  1
ANDRE GARDINAL FERREIRA DUARTE            1
ALEXANDRE OBERHERR                        1
JULIANA BARBOZA MATOS DE SOUZA            1
GEZIEL JANUARIO DA SILVA                  1
MARIA IZABEL DE ANDRADE GOMES             1
ALVARO CARVALHO MACHADO                   1
JOSE IVO VANNUCHI                         1
MARCIA VALESKA FERNANDES GENEROSO         1
MARCOS ANTONIO ASSIS SALVADOR             1
KESIA DE ARAUJO SEIGNEMARTIN              1
ESMERALDA HERNANDEZ DE SOUZA              1
AUGUSTO HIDEATO CIMINO TAKEDA             1
GERALDO TELES MACHADO JUNIOR              1
GIVALDO DOS SANTOS GONCALVES              1
ANDRE ABEL DA SILVA                       1
GILVAN RIBEIRO DOS SANTOS                 1
FREDERICO MARCIO MASSANORI HIGUCHI        1
LUIZ ANTONIO CAETANO RODRIGUES            1
JOAO AUGUSTO DO CARMO           

# Cruza remuneração e lotação

In [88]:
servidores_gabinetes = pd.merge(df_pagamentos, gabinetes, left_on='nome_func', right_on='nome_func')

In [89]:
servidores_gabinetes.reset_index().head()

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
0,0,2015,01,ABNEIA BUFFOLIN BROCANELLI,"4.599,47","3.758,26","841,21",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
1,1,2015,02,ABNEIA BUFFOLIN BROCANELLI,"1.533,16","1.370,40","162,76",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
2,2,2015,03,ABNEIA BUFFOLIN BROCANELLI,"0,00","0,00","0,00",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
3,3,2015,04,ABNEIA BUFFOLIN BROCANELLI,"0,00","-7,43","7,43",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
4,4,2015,05,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.348,13","992,44",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO


# Verifica nomes não encontrados na lista de remunerações - apenas 1 deles não é comissionado <br>
# Ou seja, são servidores que não têm o salário divulgado

In [41]:
nomes_naoencontrados = pd.merge(df_pagamentos,
                        gabinetes,
                        left_on='nome_func', 
                        right_on='nome_func',
                        how='outer',
                        indicator=True)

rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

In [42]:
rdf['nome_func'].value_counts()

GABRIEL FERREIRINHA TESTA                       1
AMAURI PEGORARO                                 1
CAROLINA VIEIRA DA SILVA                        1
MARTA JANETE GARCIA DE SOUZA                    1
FABIO ADONIS PEREIRA                            1
FRANK MARQUES JUNIOR                            1
MARCELO FERREIRA                                1
RODRIGO ANTUNES BARBOSA                         1
CLAUDIO ARAUJO MACHADO                          1
MARIA DA GRACA OLBRICH ZANELATO BUCHI COELHO    1
DAVI AUGUSTO MINARDI DOS SANTOS                 1
MARCOS FARIA                                    1
FERNANDO VINICIUS PERAMA COSTA                  1
TARCISIO DANIEL BOAVENTURA                      1
SAMIA MALUF                                     1
MAURO ABALEN DE SANT'ANA                        1
ANA CLAUDIA GALVAO SILVA                        1
ANTONIO HENRIQUE ASSIS BRAZAO DE SOUZA          1
VICTOR RAMALHOSO GUERREIRO                      1
RODRIGO DE OLIVEIRA                             1


In [43]:
rdf['tipo_cargo2'].value_counts()

COMISSÃO                                                                    90
COMISSIONAMENTO SEM PREJUÍZO                                                 4
COMISSÃO\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t/ EFETIVO                          1
COMISSIONAMENTO SEM PREJUÍZO\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t/ COMISSÃO     1
EFETIVO                                                                      1
Name: tipo_cargo2, dtype: int64

In [44]:
nome = rdf[(rdf['tipo_cargo2'] == 'EFETIVO')]
nome

,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
212238,NaN,NaN,LUIZ CARLOS DE ALMEIDA PRADO ALVES FILHO,NaN,NaN,NaN,GABINETE DEP. FERNANDO CURY,TÉCNICO LEGISLATIVO\n\t\t\t\t\t\t\t\n\t\t\t\t\...,EFETIVO


In [45]:
rdf.to_excel('resultados/servidores_naoencontrados.xlsx',sheet_name='Sheet1')

In [90]:
servidores_gabinetes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66468 entries, 0 to 66467
Data columns (total 9 columns):
ano                    66468 non-null object
mes                    66468 non-null object
nome_func              66468 non-null object
remuneracao_bruta      66468 non-null object
remuneracao_liquida    66468 non-null object
tributos               66468 non-null object
lotacao                66468 non-null object
tipo_cargo1            66468 non-null object
tipo_cargo2            66468 non-null object
dtypes: object(9)
memory usage: 5.1+ MB


In [91]:
servidores_gabinetes.reset_index().head(10)

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
0,0,2015,01,ABNEIA BUFFOLIN BROCANELLI,"4.599,47","3.758,26","841,21",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
1,1,2015,02,ABNEIA BUFFOLIN BROCANELLI,"1.533,16","1.370,40","162,76",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
2,2,2015,03,ABNEIA BUFFOLIN BROCANELLI,"0,00","0,00","0,00",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
3,3,2015,04,ABNEIA BUFFOLIN BROCANELLI,"0,00","-7,43","7,43",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
4,4,2015,05,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.348,13","992,44",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
5,5,2015,06,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
6,6,2015,07,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
7,7,2015,08,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
8,8,2015,09,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
9,9,2015,10,ABNEIA BUFFOLIN BROCANELLI,"5.340,57","4.357,48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO


Transforma valores para padrão decimal que permite operações matemáticas

In [92]:
def limpeza1(linha):
    valor = linha["remuneracao_bruta"]
    valor = valor.replace('.', '')
    valor = valor.replace(',', '.')
    return valor

def limpeza2(linha):
    valor = linha["remuneracao_liquida"]
    valor = valor.replace('.', '')
    valor = valor.replace(',', '.')
    return valor

In [93]:
servidores_gabinetes["remuneracao_bruta"] = servidores_gabinetes.apply(limpeza1, axis=1)
servidores_gabinetes["remuneracao_liquida"] = servidores_gabinetes.apply(limpeza2, axis=1)

servidores_gabinetes['remuneracao_bruta'] = servidores_gabinetes.remuneracao_bruta.astype('float64')
servidores_gabinetes['remuneracao_liquida'] = servidores_gabinetes.remuneracao_liquida.astype('float64')

In [94]:
servidores_gabinetes.reset_index().head(10)

,index,ano,mes,nome_func,remuneracao_bruta,remuneracao_liquida,tributos,lotacao,tipo_cargo1,tipo_cargo2
0,0,2015,01,ABNEIA BUFFOLIN BROCANELLI,"4,599.47","3,758.26","841,21",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
1,1,2015,02,ABNEIA BUFFOLIN BROCANELLI,"1,533.16","1,370.40","162,76",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
2,2,2015,03,ABNEIA BUFFOLIN BROCANELLI,0.00,0.00,"0,00",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
3,3,2015,04,ABNEIA BUFFOLIN BROCANELLI,0.00,-7.43,"7,43",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
4,4,2015,05,ABNEIA BUFFOLIN BROCANELLI,"5,340.57","4,348.13","992,44",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
5,5,2015,06,ABNEIA BUFFOLIN BROCANELLI,"5,340.57","4,357.48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
6,6,2015,07,ABNEIA BUFFOLIN BROCANELLI,"5,340.57","4,357.48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
7,7,2015,08,ABNEIA BUFFOLIN BROCANELLI,"5,340.57","4,357.48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
8,8,2015,09,ABNEIA BUFFOLIN BROCANELLI,"5,340.57","4,357.48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO
9,9,2015,10,ABNEIA BUFFOLIN BROCANELLI,"5,340.57","4,357.48","983,09",GABINETE DEP. CELSO NASCIMENTO,ASSISTENTE PARLAMENTAR V\n\t\t\t\t\t\t\t\n\t\t...,COMISSÃO


# Análise histórica de gastos com servidores por gabinete

Retira os homônimos da base, já que não é possível dizer de qual gabinete pertenciam

In [98]:
servidores_gabinetes_ajustado = servidores_gabinetes[(servidores_gabinetes['nome_func'] != 'JOAO CARLOS FRANCISCO DA SILVA') &
                                            (servidores_gabinetes['nome_func'] != 'ANA MARIA DA SILVA') &
                                            (servidores_gabinetes['nome_func'] != 'PAULO EDUARDO ALVES FERREIRA') 
                           ].copy()

Gabinetes com gastos

In [99]:
a = len(servidores_gabinetes_ajustado['lotacao'].value_counts())
a

102

Soma dos gastos com servidores

In [100]:
"${:,.2f}".format(servidores_gabinetes_ajustado.remuneracao_bruta.sum())

'$534,655,725.82'

Média

In [101]:
servidores_gabinetes_ajustado['remuneracao_bruta'].mean()

8100.353400096965

Mediana

In [102]:
"${:,.0f}".format(servidores_gabinetes_ajustado.remuneracao_bruta.median())

'$6,265'

In [103]:
servidores_gabinetes_ajustado.to_excel('resultados/remuneracao_servidores.xlsx',sheet_name='Sheet1')

In [104]:
servidores_gabinetes_ajustado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66004 entries, 0 to 66467
Data columns (total 9 columns):
ano                    66004 non-null object
mes                    66004 non-null object
nome_func              66004 non-null object
remuneracao_bruta      66004 non-null float64
remuneracao_liquida    66004 non-null float64
tributos               66004 non-null object
lotacao                66004 non-null object
tipo_cargo1            66004 non-null object
tipo_cargo2            66004 non-null object
dtypes: float64(2), object(7)
memory usage: 5.0+ MB


Gastos com servidores por gabinete - total mensal

In [105]:
servidores_gabinetes_ajustado.groupby("lotacao").remuneracao_bruta.sum().reset_index().sort_values("remuneracao_bruta", ascending=False)

,lotacao,remuneracao_bruta
9,GABINETE DEP. ALDO DEMARCHI,"9,127,045.19"
44,GABINETE DEP. FERNANDO CAPEZ,"8,788,304.61"
11,GABINETE DEP. ANA DO CARMO,"8,586,919.85"
61,GABINETE DEP. LUIZ CARLOS GONDIM,"8,465,435.58"
19,GABINETE DEP. CAMPOS MACHADO,"8,121,637.14"
63,GABINETE DEP. LUIZ TURCO,"7,970,191.27"
88,GABINETE DEP. ROBERTO MORAIS,"7,860,799.52"
21,GABINETE DEP. CARLOS CEZAR,"7,845,245.70"
59,GABINETE DEP. JOÃO PAULO RILLO,"7,744,789.12"
55,GABINETE DEP. JORGE CARUSO,"7,675,330.98"


Porcentagens em relação ao total - Gastos com servidores por gabinete - total mensal

In [107]:
# Agrupamento dos gastos por categoria
df_gabinete = servidores_gabinetes_ajustado.groupby(['lotacao']).agg({'remuneracao_bruta': sum})\
                                .sort_values('remuneracao_bruta', ascending = False)\
                                .reset_index().copy()

# Cálculo de porcentagem de cada categoria no total de gastos
df_gabinete['porcentagem'] = (df_gabinete['remuneracao_bruta'] / df_gabinete['remuneracao_bruta'].sum()) * 100

# Formatação de números para duas casas decimais
df_gabinete['remuneracao_bruta']['remuneracao_bruta'] = df_gabinete['remuneracao_bruta'].map('{:,.2f}'.format)
df_gabinete['porcentagem'] = df_gabinete['porcentagem'].map('{:,.2f}'.format)

# Resultado
df_gabinete.reset_index().head(10)

/home/reinaldo/Documentos/Code/volt/alesp/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,index,lotacao,remuneracao_bruta,porcentagem
0,0,GABINETE DEP. ALDO DEMARCHI,"9,127,045.19",1.71
1,1,GABINETE DEP. FERNANDO CAPEZ,"8,788,304.61",1.64
2,2,GABINETE DEP. ANA DO CARMO,"8,586,919.85",1.61
3,3,GABINETE DEP. LUIZ CARLOS GONDIM,"8,465,435.58",1.58
4,4,GABINETE DEP. CAMPOS MACHADO,"8,121,637.14",1.52
5,5,GABINETE DEP. LUIZ TURCO,"7,970,191.27",1.49
6,6,GABINETE DEP. ROBERTO MORAIS,"7,860,799.52",1.47
7,7,GABINETE DEP. CARLOS CEZAR,"7,845,245.70",1.47
8,8,GABINETE DEP. JOÃO PAULO RILLO,"7,744,789.12",1.45
9,9,GABINETE DEP. JORGE CARUSO,"7,675,330.98",1.44


Quantidade de servidores por gabinete historicamente

In [108]:
servidores_gabinetes_ajustado.groupby("lotacao").nome_func.count().sort_values(ascending=False)

lotacao
GABINETE DEP. ANA DO CARMO                         1080
GABINETE DEP. LUIZ TURCO                           1076
GABINETE DEP. JOOJI HATO                           1036
GABINETE DEP. MARCOS DAMASIO                       1035
GABINETE DEP. LECI BRANDÃO                         1015
GABINETE DEP. LUIZ CARLOS GONDIM                   1006
GABINETE DEP. CORONEL CAMILO                        989
GABINETE DEP. MARCOS MARTINS                        954
GABINETE DEP. CEZINHA DE MADUREIRA                  947
GABINETE DEP. PROFESSOR AURIEL                      946
GABINETE DEP. TEONILIO BARBA                        941
GABINETE DEP. GILENO GOMES                          935
GABINETE DEP. CARLOS NEDER                          925
GABINETE DEP. GERALDO CRUZ                          918
GABINETE DEP. JORGE CARUSO                          910
GABINETE DEP. MARTA COSTA                           880
GABINETE DEP. EDSON GIRIBONI                        875
GABINETE DEP. ENIO TATTO                

Servidores com maiores remuneração histórica

In [109]:
servidores_gabinetes_ajustado.groupby("nome_func").remuneracao_bruta.sum().reset_index().sort_values("remuneracao_bruta", ascending=False).head(10)

,nome_func,remuneracao_bruta
1855,RICARDO DE OLIVEIRA,"1,138,350.40"
696,EVALDO PEREIRA DA SILVA,"1,083,576.84"
1447,MARCOS FRANCISCO DE SOUZA,"1,083,576.84"
1389,MARCELO SOUZA SERPA,"1,083,576.84"
1126,JOSE LUIZ ALVES,"1,083,576.84"
1098,JOSE CAVALLI JUNIOR,"1,080,520.98"
1924,ROSA MARIA BUENO VELAZQUEZ,"1,074,053.72"
197,ANTONIO CARLOS DIAS PEREIRA FILHO,"1,069,247.87"
418,CLAUDIO FERREIRA,"1,061,702.43"
1271,LUCIA HELENA JUVENCIO,"1,055,913.89"


Mostra onde nome de servidor aparece ou já esteve

In [119]:
nome = df_lotacao[(df_lotacao['nome_func'] == 'RICARDO DE OLIVEIRA')].copy()
nome

,lotacao,nome_func,tipo_cargo1,tipo_cargo2
22990,GABINETE DEP. EDMIR CHEDID,RICARDO DE OLIVEIRA,ASSESSOR ESPECIAL PARLAMENTAR,COMISSÃO
22991,SERVIÇO DE ADMINISTRAÇÃO GERAL,RICARDO DE OLIVEIRA,TÉCNICO LEGISLATIVO,EFETIVO
22992,Ex-funcionário,RICARDO DE OLIVEIRA,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO


# Mostra julho 2018

In [111]:
servidores_gabinetes_ajustado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66004 entries, 0 to 66467
Data columns (total 9 columns):
ano                    66004 non-null object
mes                    66004 non-null object
nome_func              66004 non-null object
remuneracao_bruta      66004 non-null float64
remuneracao_liquida    66004 non-null float64
tributos               66004 non-null object
lotacao                66004 non-null object
tipo_cargo1            66004 non-null object
tipo_cargo2            66004 non-null object
dtypes: float64(2), object(7)
memory usage: 5.0+ MB


In [114]:
julho = servidores_gabinetes_ajustado[(servidores_gabinetes_ajustado['ano'] == '2018') &
                            (servidores_gabinetes_ajustado['mes'] == '07') 
                           ]

In [115]:
julho.groupby("lotacao").remuneracao_bruta.sum().reset_index().sort_values("remuneracao_bruta", ascending=False)

,lotacao,remuneracao_bruta
9,GABINETE DEP. ALDO DEMARCHI,"296,176.65"
11,GABINETE DEP. ANA DO CARMO,"274,280.22"
44,GABINETE DEP. FERNANDO CAPEZ,"267,229.72"
39,GABINETE DEP. EDMIR CHEDID,"262,928.55"
63,GABINETE DEP. LUIZ TURCO,"253,377.06"
91,GABINETE DEP. ROGÉRIO NOGUEIRA,"247,269.38"
61,GABINETE DEP. LUIZ CARLOS GONDIM,"240,059.15"
90,GABINETE DEP. RODRIGO MORAES,"236,039.87"
80,GABINETE DEP. RAFAEL SILVA,"231,000.95"
8,GABINETE DEP. AFONSO LOBATO,"226,592.05"


In [116]:
gastos_julho = julho.groupby("lotacao").remuneracao_bruta.sum().reset_index().sort_values("remuneracao_bruta", ascending=False)

In [118]:
gastos_julho.to_excel('resultados/remuneracao_servidores_julho.xlsx',sheet_name='Sheet1')

Servidores com maior remuneração em julho de 2018

In [120]:
julho.groupby("nome_func").remuneracao_bruta.sum().reset_index().sort_values("remuneracao_bruta", ascending=False).head(10)

,nome_func,remuneracao_bruta
1849,RICARDO DE OLIVEIRA,"30,770.62"
1808,REINALDO DA SILVA,"26,403.11"
1121,JOSE LUIZ ALVES,"25,322.25"
695,EVALDO PEREIRA DA SILVA,"25,322.25"
1513,MARIA TERESA DOS SANTOS SIQUEIRA,"25,322.25"
1470,MARIA BETANIA DE AQUINO MORAES,"25,322.25"
1469,MARIA APARECIDA JANUARIO,"25,322.25"
1442,MARCOS FRANCISCO DE SOUZA,"25,322.25"
1918,ROSA MARIA BUENO VELAZQUEZ,"25,322.25"
1093,JOSE CAVALLI JUNIOR,"25,322.25"


Mostra o gabinete do maior

In [121]:
nome = df_lotacao[(df_lotacao['nome_func'] == 'RICARDO DE OLIVEIRA')].copy()
nome

,lotacao,nome_func,tipo_cargo1,tipo_cargo2
22990,GABINETE DEP. EDMIR CHEDID,RICARDO DE OLIVEIRA,ASSESSOR ESPECIAL PARLAMENTAR,COMISSÃO
22991,SERVIÇO DE ADMINISTRAÇÃO GERAL,RICARDO DE OLIVEIRA,TÉCNICO LEGISLATIVO,EFETIVO
22992,Ex-funcionário,RICARDO DE OLIVEIRA,AGENTE DE SEGURANÇA PARLAMENTAR,COMISSÃO
